In [1]:
# imports 
import os
os.environ['picaso_refdata'] = '/home/afinemax/afinemax/khazad-dum/school_2023_2024/picaso/reference' #THIS MUST GO BEFORE YOUR IMPORT STATEMENT
os.environ['PYSYN_CDBS'] = '/home/afinemax/afinemax/khazad-dum/school_2023_2024/Pysynphot/grp/redcat/trds' #this is for the stellar data discussed below.
import picaso.justdoit as jdi
from picaso import justplotit as jpi
jpi.output_notebook()

from picaso import justdoit as jdi 
from picaso import justplotit as jpi
import numpy as np

refdata = os.getenv("picaso_refdata")
os.environ['jdi.opannection'] = 'home/afinemax/afinemax/khazad-dum/school_2023_2024/picaso'
jdi_data = os.getenv('jdi.opannection')
print(refdata)

import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

opacity = jdi.opannection(wave_range=[0.3,1]) #lets just use all defaults

from bokeh.io import export_svg

# custom color pallate 
my_colors = ['#648FFF', '#785EF0', '#DC267F', 
             '#FE6100', '#FFB000', '#48E95E', 
             '#5014C2', '#EA67A2', '#4A4952', 
             '#9A02D9', '#590D38', '#A9F3EE',
             '#E894F1', '#8F1B0B', '#EC6E0C']


/home/afinemax/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading BokehJS ...

/home/afinemax/afinemax/khazad-dum/school_2023_2024/picaso/reference


# Reflected light spectra of cool planets

1. Following the tutorial “Basics of Reflected Light”, plot the albedo spectrum of a
Jupiter-like atmosphere (without any clouds for now) in the wavelength range 0.3 - 1.0
micron. Vary the spectral resolution (R) between 50 and 10,000. What qualitative
changes in the spectrum do you notice when you increase the spectral resolution? What
does that tell you about the nature of the main sources of opacity in this atmosphere?
2. Investigate which molecules (among those commonly found in the atmospheres of cold
giant planets, e.g. water, methane, ammonia, carbon monoxide, carbon dioxide, …) are
the dominant sources of opacity for your model atmosphere, in the wavelength range
specified above?
3. Add simple clouds to your model atmosphere (see the tutorial “Adding Cloud Opacity,
Scattering, Asymmetry”) by specifying the properties of the cloud layer, i.e. its
asymmetry factor, single scattering albedo, opacity, pressure level, and thickness.
1
Discuss the meaning (definition) of each of these quantities. Investigate how they affect
the resulting spectra by varying their values.
4. Are there any degeneracies between different model parameters? In other words, is it
possible to obtain similar spectra for different combinations of parameters or does each
parameter leave a unique signature in the resulting spectrum? Briefly discuss the
implications of your findings.

### 1. 
Following the tutorial “Basics of Reflected Light”, plot the albedo spectrum of a
Jupiter-like atmosphere (without any clouds for now) in the wavelength range 0.3 - 1.0
micron. Vary the spectral resolution (R) between 50 and 10,000. What qualitative
changes in the spectrum do you notice when you increase the spectral resolution? What
does that tell you about the nature of the main sources of opacity in this atmosphere?

In [2]:
np.arange(50, 10000, 5)

array([  50,   55,   60, ..., 9985, 9990, 9995])

In [3]:
opacity = jdi.opannection(wave_range=[0.3,1]) # micrometer
start_case = jdi.inputs() # load blank slate

# Set Planet & Star properties
#phase angle
start_case.phase_angle(0) #radians

#define gravity
start_case.gravity(gravity=25, gravity_unit=jdi.u.Unit('m/(s**2)')) #any astropy units available

#define star
start_case.star(opacity, 5000,0,4.0) #opacity db, pysynphot database, temp, metallicity, logg

# Set Atm conditions, 2 options
# 1) Specifying a file path to model run
# 2) Give arbitrary pressure, temperature and composition directly as a dictionary input

# We will go with the jupiter db file, as we are modleing a Jpiter-like atmosphere
print(jdi.jupiter_pt()) #should return the path to your reference data
start_case.atmosphere(filename=jdi.jupiter_pt(), delim_whitespace=True)


# File Format, if specifying from file
# Must have pressure(bars), temperature(Kelvin), and case sensitive molecule names (e.g. TiO, Na, H2O, etc) for mixing ratios (in no particular order)
# Can specify any necessary key word arguments for pd.read_csv at the end

# print out first few lines of file
comp_file = jdi.pd.read_csv(jdi.jupiter_pt(), delim_whitespace=True)
#see example below
comp_file.head()


# First spectrum 
df = start_case.spectrum(opacity)
print('First Spectrum')
print('Checkout out what was returned (Note this is a change in v1.0)')
print(df.keys())
print(f'''FpFs is the relative flux of the planet and star or: 
$\\frac{{f_p}}{{f_s}} = a_{{\lambda}} \\left(\\frac{{r_p}}{{a}}\\right)^2$''')

# cia is collision induced asbsorbtion 
# gather outputs
wno, alb, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']

for R in np.linspace(50, 10000, 5):
    wno, alb, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
    wno, alb = jdi.mean_regrid(wno, alb , R=R) # this is my R that we are changing 
    jpi.show(jpi.spectrum(wno, alb, plot_width=500,x_range=[0.3,1])) # plot


# Create a jpi figure
wno, alb, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
wno, alb = jdi.mean_regrid(wno, alb , R=50, )
fig = jpi.spectrum(wno, alb, plot_width=750, x_range=[0.3,1], title='Albedo Spectrum', )

# Iterate over R values and plot the lines
R_values = np.linspace(50, 10000, 3)

for i in range(len(R_values)):
    R = R_values[-i]
    wno, alb, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
    wno, alb = jdi.mean_regrid(wno, alb , R=R, ) # this is my R that we are changing 
    fig.line(1e4/wno, alb, legend_label=f'R={R:.2f}', line_width=2, line_alpha = 0.7, color = my_colors[i])

# Show the plot
jpi.show(fig)



/home/afinemax/afinemax/khazad-dum/school_2023_2024/picaso/reference/base_cases/jupiter.pt
First Spectrum
Checkout out what was returned (Note this is a change in v1.0)
dict_keys(['wavenumber', 'albedo', 'bond_albedo', 'fpfs_reflected'])
FpFs is the relative flux of the planet and star or: 
$\frac{f_p}{f_s} = a_{\lambda} \left(\frac{r_p}{a}\right)^2$


###  2. 
Investigate which molecules (among those commonly found in the atmospheres of cold
giant planets, e.g. water, methane, ammonia, carbon monoxide, carbon dioxide, …) are
the dominant sources of opacity for your model atmosphere, in the wavelength range
specified above?

In [4]:
start_case.atmosphere(filename=jdi.jupiter_pt(), exclude_mol=['CH4', 'NH3', 'CO', 'CO2'], delim_whitespace=True) # model of jupiter atm - specified species 

df = start_case.spectrum(opacity) # model of jupiters atm
wno_ch4, alb_ch4, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']

wno_nowater, alb_nowater, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
wno_nowater, alb_nowater= jdi.mean_regrid(wno_ch4, alb_ch4 , R=150)
fig = jpi.spectrum(wno, alb, plot_width=750, title='Albedo Spectrum: Varying Composition')
fig.line(1e4/wno_nowater, alb_nowater, line_width=2, color='#A8EF2C', line_alpha = 0.7, legend_label='No : CH4, NH3, CO, CO2')

start_case.atmosphere(filename=jdi.jupiter_pt(), exclude_mol=['H2O'], delim_whitespace=True) # model of jupiter atm - specified species 
df = start_case.spectrum(opacity) # model of jupiters atm
wno_ch4, alb_ch4, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
wno_nowater, alb_nowater, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
wno_nowater, alb_nowater= jdi.mean_regrid(wno_ch4, alb_ch4 , R=150)
fig.line(1e4/wno_nowater, alb_nowater, line_width=2, color='#8544D0', line_alpha = 0.7, legend_label ='No: H2O')




jpi.show(fig)

In [5]:
start_case.atmosphere(filename=jdi.jupiter_pt(), exclude_mol=['CH4', 'NH3', 'CO', 'CO2'], delim_whitespace=True) # model of jupiter atm - specified species 

df = start_case.spectrum(opacity) # model of jupiters atm
wno_ch4, alb_ch4, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']

wno_nowater, alb_nowater, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
wno_nowater, alb_nowater= jdi.mean_regrid(wno_ch4, alb_ch4 , R=150)
fig = jpi.spectrum(wno, alb, plot_width=750, title='test title',)
fig.line(1e4/wno_nowater, alb_nowater, line_width=2, color='red')


jpi.show(fig)

In [6]:
start_case.atmosphere(filename=jdi.jupiter_pt(), exclude_mol=['H2O'], delim_whitespace=True) # model of jupiter atm - specified species 

df = start_case.spectrum(opacity) # model of jupiters atm
wno_ch4, alb_ch4, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']

wno_nowater, alb_nowater, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
wno_nowater, alb_nowater= jdi.mean_regrid(wno_ch4, alb_ch4 , R=150)
fig = jpi.spectrum(wno, alb, plot_width=750)
fig.line(1e4/wno_nowater, alb_nowater, line_width=2, color='red')


jpi.show(fig)

In [7]:
# appear to show moleculer lines at higher R
# H2O looks to be the most signifiant cause of opacities from NH3, CH4, CO, CO2


### 3. 
Add simple clouds to your model atmosphere (see the tutorial “Adding Cloud Opacity,
Scattering, Asymmetry”) by specifying the properties of the cloud layer, i.e. its
asymmetry factor, single scattering albedo, opacity, pressure level, and thickness.
1
Discuss the meaning (definition) of each of these quantities. Investigate how they affect
the resulting spectra by varying their values.


### Definitions:
asymmetry factor - 

single scattering albedo -

opacity -

pressure level -

hickness -

In [8]:
opa = jdi.opannection(wave_range=[0.3,1])
case1 = jdi.inputs()

#phase
case1.phase_angle(0) #radians

#gravity
case1.gravity(gravity = 25, gravity_unit=jdi.u.Unit('m/(s**2)'))

#star 
# this is different from above liar
case1.star(opa, 6000,0.0122,4.437) #kelvin, log metal, log cgs

#atmosphere
case1.atmosphere(filename= jdi.jupiter_pt(), delim_whitespace=True)


colors = ['skyblue','firebrick','olivedrab']
case1.clouds(g0=[0.9], w0=[0.8], opd=[0.5], p=[0.0], dp=[1.0])  # Slab cloud from 1.0 bar up to 0.1 bar
df = case1.spectrum(opa)
wno, alb, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
wno, alb = jdi.mean_regrid(wno, alb, R=150)
fig = jpi.spectrum(wno, alb, plot_width=1600, plot_height=1000)

#set model clouds, note these are lists since you can specify multiple cloud layers
#g0 : Asymmetry factor of the cloud (value between 0 - 1)
#w0 : Single scattering albedo (value between 0 - 1)
#opd : Total extinction (tau) of each layer (not cumulative)

for i, g0 in enumerate([0.1,0.5,0.9]):
    print([g0])
    case1.clouds(g0=[g0], w0=[0.8], opd=[0.5], p=[0.0], dp=[1.0])  # Slab cloud from 1.0 bar up to 0.1 bar
    df = case1.spectrum(opa)
    wno, alb, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
    wno, alb = jdi.mean_regrid(wno, alb, R=150)
    fig.line(1e4/wno, alb, line_width=3, color=my_colors[i], legend_label=f'g0={g0} ')
    
    
for i, w0 in enumerate([0.1,0.5,0.9]):
    case1.clouds(g0=[0.8], w0=[w0], opd=[0.5], p=[0.0], dp=[1.0])  # Slab cloud from 1.0 bar up to 0.1 bar
    df = case1.spectrum(opa)
    wno, alb, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
    wno, alb = jdi.mean_regrid(wno, alb, R=150)
    fig.line(1e4/wno, alb, line_width=3, color=my_colors[i + 3], legend_label=f'w0={w0} ')
    
for i, opd in enumerate([0.1,0.5,0.9]):
    case1.clouds(g0=[0.8], w0=[0.8], opd=[opd], p=[0.0], dp=[1.0])  # Slab cloud from 1.0 bar up to 0.1 bar
    df = case1.spectrum(opa)
    wno, alb, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
    wno, alb = jdi.mean_regrid(wno, alb, R=150)
    fig.line(1e4/wno, alb, line_width=3, color=my_colors[i + 6], legend_label=f'opd={opd} ')
    
for i, p in enumerate([3,0,-3]):
    case1.clouds(g0=[0.8], w0=[0.8], opd=[0.5], p=[p], dp=[1.0])  # Slab cloud from 1.0 bar up to 0.1 bar
    df = case1.spectrum(opa)
    wno, alb, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
    wno, alb = jdi.mean_regrid(wno, alb, R=150)
    fig.line(1e4/wno, alb, line_width=3, color = my_colors[i + 9],legend_label=f'p={p} ')
    
    
for i, dp in enumerate([0.1,0.5,0.9]):
    case1.clouds(g0=[0.8], w0=[0.8], opd=[0.5], p=[0.0], dp=[dp])  # Slab cloud from 1.0 bar up to 0.1 bar
    df = case1.spectrum(opa)
    wno, alb, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
    wno, alb = jdi.mean_regrid(wno, alb, R=150)
    fig.line(1e4/wno, alb, line_width=3, color = my_colors[i + 12],legend_label=f'dp={dp} ')
    
     
    

jpi.show(fig)

[0.1]
[0.5]
[0.9]


In [9]:
opa = jdi.opannection(wave_range=[0.3,1])
case1 = jdi.inputs()

#phase
case1.phase_angle(0) #radians

#gravity
case1.gravity(gravity = 25, gravity_unit=jdi.u.Unit('m/(s**2)'))

#star 
# this is different from above liar
case1.star(opa, 6000,0.0122,4.437) #kelvin, log metal, log cgs

#atmosphere
case1.atmosphere(filename= jdi.jupiter_pt(), delim_whitespace=True)


colors = ['skyblue','firebrick','olivedrab']
case1.clouds(g0=[0.9], w0=[0.8], opd=[0.5], p=[0.0], dp=[1.0])  # Slab cloud from 1.0 bar up to 0.1 bar
df = case1.spectrum(opa)
wno, alb, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
wno, alb = jdi.mean_regrid(wno, alb, R=150)
fig = jpi.spectrum(wno, alb, plot_width=500)

#set model clouds, note these are lists since you can specify multiple cloud layers
#g0 : Asymmetry factor of the cloud (value between 0 - 1)
#w0 : Single scattering albedo (value between 0 - 1)
#opd : Total extinction (tau) of each layer (not cumulative)

for i, w0 in enumerate([0.1,0.5,0.9]):
    case1.clouds(g0=[0.8], w0=[w0], opd=[0.5], p=[0.0], dp=[1.0])  # Slab cloud from 1.0 bar up to 0.1 bar
    df = case1.spectrum(opa)
    wno, alb, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
    wno, alb = jdi.mean_regrid(wno, alb, R=150)
    fig.line(1e4/wno, alb, line_width=3, color=my_colors[i + 2], legend_label=f'w0={w0} ')
    
    

jpi.show(fig)


In [10]:
# Same inputs as before
opa = jdi.opannection(wave_range=[0.3,1])
case1 = jdi.inputs()

#phase
case1.phase_angle(0) #radians

#gravity
case1.gravity(gravity = 25, gravity_unit=jdi.u.Unit('m/(s**2)'))

#star 
# this is different from above liar
case1.star(opa, 6000,0.0122,4.437) #kelvin, log metal, log cgs

#atmosphere
case1.atmosphere(filename= jdi.jupiter_pt(), delim_whitespace=True)

# define cloud jargon 
print( jdi.jupiter_cld())

#set model clouds
case1.clouds( filename= jdi.jupiter_cld(), delim_whitespace=True)


#run
df = case1.spectrum(opa)
wno, alb, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
wno, alb = jdi.mean_regrid(wno, alb, R=150)

jpi.show(jpi.spectrum(wno, alb, plot_width=750))


/home/afinemax/afinemax/khazad-dum/school_2023_2024/picaso/reference/base_cases/jupiterf3.cld


In [11]:
# Same inputs as before
opa = jdi.opannection(wave_range=[0.3,1])
case1 = jdi.inputs()

#phase
case1.phase_angle(0) #radians

#gravity
case1.gravity(gravity = 25, gravity_unit=jdi.u.Unit('m/(s**2)'))

#star 
# this is different from above liar
case1.star(opa, 6000,0.0122,4.437) #kelvin, log metal, log cgs

#atmosphere
case1.atmosphere(filename= jdi.jupiter_pt(), delim_whitespace=True)

# define cloud jargon 
print( jdi.jupiter_cld())


#set model clouds, note these are lists since you can specify multiple cloud layers
#g0 : Asymmetry factor of the cloud (value between 0 - 1)
#w0 : Single scattering albedo (value between 0 - 1)
#opd : Total extinction (tau) of each layer (not cumulative)

my_g0 = [0, 0,]
my_w0 = [ 1, 0, ]
my_opd = [0.01, 1, 0.5]
my_p = [0, 0, 0.5, ]

# p is the maximum cloud pressure in log. Max Pressure = 10^p bars
# dp is the thickness in log. Minimum Pressure = 10^(p-dp) bars

#case1.clouds( g0=[0.9], w0=[0.8], opd=[0.5], p = [0.0], dp=[1.0])  # Slab cloud from 1.0 bar up to 0.1 bar

for i in range(len(my_g0)):
    g0 = my_g0[i]
    w0 = my_w0[i]
    opd = my_opd[i]
    p = my_p[i]
    case1.clouds( g0=[g0], w0=[w0], opd=[opd], p = [p], dp=[1.0])


    #run
    df = case1.spectrum(opa)
    wno, alb, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
    wno, alb = jdi.mean_regrid(wno, alb, R=150)

    #fig = jpi.spectrum(wno, alb, plot_width=750)
    #fig.line(1e4/wno_nowater, alb_nowater, line_width=2, color='red')
    #jpi.show(fig)

    jpi.show(jpi.spectrum(wno, alb, plot_width=750))





/home/afinemax/afinemax/khazad-dum/school_2023_2024/picaso/reference/base_cases/jupiterf3.cld


### 4. 
Are there any degeneracies between different model parameters? In other words, is it
possible to obtain similar spectra for different combinations of parameters or does each
parameter leave a unique signature in the resulting spectrum? Briefly discuss the
implications of your findings.

### Answer

# Transmission spectra of warm planets

5. Following the tutorial “Computing Transmission Spectroscopy”, plot the transmission
spectrum for a hot Jupiter model atmosphere in the wavelength range accessible with
the NIRSpec instrument onboard the James Webb Space Telescope (0.6 - 5 micron).

In [12]:
# We will use new Hot Jupiter template to guide us through the exercise.
opa = jdi.opannection(wave_range=[0.6,5]) # micron 

case1 = jdi.inputs()

case1.phase_angle(0)


#here we are going to have to specify gravity through R and M since we need it in the Flux calc
case1.gravity(mass=1, mass_unit=jdi.u.Unit('M_jup'),
              radius=1.2, radius_unit=jdi.u.Unit('R_jup'))

#here we are going to have to specify R as well
# I think different sun, maybe an example of one with a hot jupiter
case1.star(opa, 4000,0.0122,4.437,radius=0.7, radius_unit = jdi.u.Unit('R_sun') )


#atmo
case1.atmosphere(filename = jdi.HJ_pt(), delim_whitespace=True)

# The reference pressure sets the level of your radius. It is needed to compute the altitude and altitude-dependent gravity. 
# PICASO default is 1 bar but for gas giant calculations its more common to use 10 bar. This is easy to swap. 
# If you are running terrestrial planet models you want your reference pressure to be the highest pressure in your user-specified grid.

case1.approx(p_reference=10) # bar

df= case1.spectrum(opa, full_output=True,calculation='transmission') #note the new last key

wno, rprs2  = df['wavenumber'] , df['transit_depth']
wno, rprs2 = jdi.mean_regrid(wno, rprs2, R=150)
full_output = df['full_output']

# Analyzing Transit Spectroscopy Output¶
# plots 
jpi.show(jpi.pt(full_output,plot_width=750))
jpi.show(jpi.mixing_ratio(full_output,plot_width=750))

# plot spectrum 
# Standard (Rp/Rs)^2
jpi.show(jpi.spectrum(wno,rprs2*1e6,plot_width=750), title='Tramission Spectrum')



### 6. 
Investigate which atomic and molecular species contribute significantly to the opacity of
your model atmosphere in the wavelength range of NIRSpec. Common species include
water, carbon monoxide, carbon dioxide, methane, sodium, potassium, etc., but feel free
to explore other species as well. Discuss your findings

In [13]:
# See Molecular Contribution of Spectra
# atmo
w,f,l =[],[],[]
# water, carbon monoxide, carbon dioxide, methane, sodium, potassium
for iex in ['CH4','H2O','CO2','CO', 'K', 'Na', 'O2', 'O3' 'H2', None]:
    case1.atmosphere(filename = jdi.HJ_pt(),exclude_mol=iex, delim_whitespace=True)
    df= case1.spectrum(opa, full_output=True,calculation='transmission') #note the new last key
    wno, rprs2  = df['wavenumber'] , df['transit_depth']
    wno, rprs2 = jdi.mean_regrid(wno, rprs2, R=150)
    w +=[wno]
    f+=[rprs2]
    if iex==None:
        leg='all'
    else:
        leg = f'No {iex}'
    l+=[leg]
jpi.show(jpi.spectrum(w,f,legend=l))

### 7. 
Vary the surface gravity of your planet. What effect does this have on the resulting
spectrum? Discuss the physical reasons behind this.


### Change into units of surface gravity / average density

In [14]:
# See Molecular Contribution of Spectra
# atmo
w, f, l = [], [], []

# water, carbon monoxide, carbon dioxide, methane, sodium, potassium
for mass in [1, 2, 0.5]:
    case1.gravity(mass=mass, mass_unit=jdi.u.Unit('M_jup'),
                  radius=1.2, radius_unit=jdi.u.Unit('R_jup'))
    
    df = case1.spectrum(opa, full_output=True, calculation='transmission')  # note the new last key
    wno, rprs2 = df['wavenumber'], df['transit_depth']
    wno, rprs2 = jdi.mean_regrid(wno, rprs2, R=150)
    w += [wno]
    f += [rprs2]

    if mass is None:
        leg = 'all'
    else:
        jupiter = 'jupiter'
        leg = r'g = ' + repr(mass) + r'gJ'
    l += [leg]

jpi.show(jpi.spectrum(w, f, legend=l))


## 8. 
Add a simple model of grey (wavelength independent) clouds to your atmosphere. What
is the effect of clouds on the mean transit depth and the amplitude of spectral features?

In [15]:
# Adding Grey Cloud
case1.clouds(p = [1], dp=[4], opd=[1], g0=[0], w0=[0])

df_gry= case1.spectrum(opa, full_output=True,calculation='transmission') #note the new last key

wno, rprs2_gry  = df_gry['wavenumber'] , df_gry['transit_depth']
wno, rprs2_gry = jdi.mean_regrid(wno, rprs2_gry, R=150)
full_output_gry = df_gry['full_output']

jpi.show(jpi.spectrum([wno, wno],[1e6*(rprs2-np.mean(rprs2))
                              , (rprs2_gry-np.mean(rprs2_gry))*1e6],
                  legend=['Cloud free','Grey cloud'],
                  plot_width=750))

### 8 Comments

# Thermal emission spectra of warm planets

### 9. 
Following the tutorial “Computing Thermal Fluxes”, plot the thermal emission spectrum
for a hot Jupiter model atmosphere in the NIRSpec wavelength range.

In [16]:
# Computing thermal flux is pretty much the same deal as the reflected light.
# There are just a couple tweaks we will have to make in order to make sure we have the full set of info.


# Hot jupiter template
opa = jdi.opannection(wave_range=[0.6,5]) # micron
case1 = jdi.inputs()
case1.phase_angle(0)
#here we are going to have to specify gravity through R and M since we need it in the Flux calc
case1.gravity(mass=1, mass_unit=jdi.u.Unit('M_jup'),
              radius=1.2, radius_unit=jdi.u.Unit('R_jup'))

#here we are going to have to specify R as well
case1.star(opa, 4000,0.0122,4.437,radius=0.7, radius_unit = jdi.u.Unit('R_sun') )

#atmo
case1.atmosphere(filename = jdi.HJ_pt(), delim_whitespace=True)#

#can uncomment for clouds to explore!
#case1.clouds(filename = jdi.HJ_cld(), delim_whitespace=True)

# full ouput 
df= case1.spectrum(opa, full_output=True,calculation='thermal',) #note the new last key

wno, fpfs , fp = df['wavenumber'] , df['fpfs_thermal'], df['thermal']
wno_bin, fpfs_bin = jdi.mean_regrid(wno, fpfs, R=150)
wno_bin, fp_bin = jdi.mean_regrid(wno, fp, R=150)
full_output = df['full_output']

###########################
# plot
jpi.show(jpi.row(
    jpi.mixing_ratio(full_output, plot_height=500),
    jpi.pt(full_output, plot_height=500)))
#can also input any key word argument acceptable for bokeh.figure:
#show(jpi.mixing_ratio(full_output, plot_width=500, y_axis_type='linear',y_range=[10,1e-3]))


# spectrum plot
jpi.show(jpi.spectrum(wno_bin,fpfs_bin*1e6,plot_width=750,y_axis_type='log'))

### 10.
To first order, the emission spectrum looks like a blackbody spectrum with some spectral
features (absorption or emission lines) added. What blackbody temperature (or a
temperature range) provides a reasonably good match to the underlying continuum of
your spectrum? At what pressure level is this temperature reached in your model?

In [17]:
jpi.show(jpi.flux_at_top(df, pressures=[1e-1, 1e-2, 1e-3, 1e-4,],
                         R=150,  plot_width=750, plot_height=400))


## 11. 

Plot the relative abundance (i.e. mixing ratios) of different species as a function of
pressure level in the atmosphere. 
What is the composition of your atmosphere at the
pressure level you identified in the previous section? Describe how the composition
changes as a function of altitude.